In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

#constants
DATA_PATH = Path("../dataset/train.csv")
CLEANED_DATA_PATH = Path("../dataset/cleaned_train.csv")

In [3]:
def load_raw_data():
    """Load raw data from CSV file."""
    df = pd.read_csv(DATA_PATH)
    print(f"Loaded raw data: {df.shape}")
    return df

df_raw = load_raw_data()
df = df_raw.copy()    

Loaded raw data: (1460, 81)


In [17]:
def analyze_missing_values(df):
    """Analyze missing values in the DataFrame."""
    missing = df.isnull().sum()
    missing_percent = (missing/len(df)) * 100

    missing_df = pd.DataFrame({
        'missing_count' : missing,
        'missing_percent' : missing_percent,
        'missing_dtype' : df.dtypes
    })

    missing_df = missing_df[missing_df['missing_count']>0].sort_values(by='missing_count', ascending=False)

    return missing_df

missing_values = analyze_missing_values(df)
print("Missing values analysis:")
print(missing_values)

Missing values analysis:
              missing_count  missing_percent missing_dtype
PoolQC                 1453        99.520548        object
MiscFeature            1406        96.301370        object
Alley                  1369        93.767123        object
Fence                  1179        80.753425        object
MasVnrType              872        59.726027        object
FireplaceQu             690        47.260274        object
LotFrontage             259        17.739726       float64
GarageType               81         5.547945        object
GarageYrBlt              81         5.547945       float64
GarageFinish             81         5.547945        object
GarageQual               81         5.547945        object
GarageCond               81         5.547945        object
BsmtExposure             38         2.602740        object
BsmtFinType2             38         2.602740        object
BsmtQual                 37         2.534247        object
BsmtCond                 37    

In [19]:
def handle_missing_values(df):
    """Handle missing values in the DataFrame."""
    
    # Create a copy to avoid modifying original
    df = df.copy()
    
    # LotFrontage → median per Neighborhood
    if "LotFrontage" in df.columns:
        df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(
            lambda x: x.fillna(x.median())
        )
    
    # Numerical → median
    num_cols = df.select_dtypes(include=[np.number]).columns    
    for col in num_cols:
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].median())

    # Absence categories → fill with 'NA'/'None'
    absence_features = [
    "MasVnrType","BsmtQual","BsmtCond","BsmtExposure",
    "BsmtFinType1","BsmtFinType2","GarageType","GarageFinish",
    "GarageQual","GarageCond","FireplaceQu",
    "PoolQC","Fence","Alley","MiscFeature"]
    
    for col in absence_features:
        if col in df.columns:
            fill_value = 'None' if col == "MasVnrType" else "NA"
            df[col] = df[col].fillna(fill_value)  # Removed inplace=True

    # Electrical → mode
    if "Electrical" in df.columns and df['Electrical'].isnull().any():
        df["Electrical"] = df["Electrical"].fillna(df["Electrical"].mode()[0])  # Removed inplace=True

    # Get all categorical columns as a set
    all_cat_cols = set(df.select_dtypes(include=["object"]).columns)
    
    # Remove already handled columns using set difference
    remaining_cats = all_cat_cols - set(absence_features) - {"Electrical"}
    
    # Handle only the remaining categorical columns
    for col in remaining_cats:
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].mode()[0])
    
    return df

In [20]:
df = handle_missing_values(df)
df.isnull().sum().sum()

np.int64(0)

In [22]:
numeric_columns = ['LotFrontage', 'LotArea', 'MasVnrArea', 'GarageYrBlt']
year_columns = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']
def fix_datatypes(df):
    """Convert columns to appropriate datatypes"""
    # Convert numeric columns that were read as object
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    #convert year columns to datetime
    for col in year_columns:
        df[col] = pd.to_datetime(df[col], format="%Y", errors='coerce')
    
    return df

In [23]:
# Apply data type fixes
df = fix_datatypes(df)

# Verify changes
print("\nData Types After Conversion:")
print(df[numeric_columns + year_columns].dtypes)


Data Types After Conversion:
LotFrontage            float64
LotArea                  int64
MasVnrArea             float64
GarageYrBlt     datetime64[ns]
YearBuilt       datetime64[ns]
YearRemodAdd    datetime64[ns]
GarageYrBlt     datetime64[ns]
YrSold          datetime64[ns]
dtype: object


In [9]:
OUTLIER_COLUMNS = [
    'LotFrontage', 'LotArea', 'MasVnrArea',
    'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF',
    '1stFlrSF', '2ndFlrSF', 'GrLivArea',
    'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
    'SalePrice'
]

def analyze_outliers(df, OUTLIER_COLUMNS, n_std=3):

    outliers_info = {}

    for col in OUTLIER_COLUMNS:
        mean = df[col].mean()
        std = df[col].std()
        z_score = np.abs((df[col]-mean)/std)

        outliers = df[z_score > n_std]

        if len(outliers) > 0:
            outliers_info[col]={
                "count": len(outliers),
                "percent": (len(outliers) / len(df)) * 100,
                "min": df[col].min(),
                "max": df[col].max(),
                "threshold_low": mean - n_std * std,
                "threshold_high": mean + n_std * std
            }
    
    return pd.DataFrame(outliers_info).T

In [14]:
# Define custom thresholds for specific columns
COLUMN_THRESHOLDS = {
    'LotArea': 2.5,        # More strict for lot area
    'SalePrice': 2.5,      # More strict for price
    'GrLivArea': 2.5,      # More strict for living area
    'default': 3           # Default for other columns
}

def handling_outliers(df, columns, thresholds=COLUMN_THRESHOLDS, method='clip'):
    df = df.copy()
    
    for col in columns:
        n_std = thresholds.get(col, thresholds['default'])
        mean = df[col].mean()
        std = df[col].std()
        
        if method == 'clip':
            df[col] = df[col].clip(
                lower=mean - n_std * std,
                upper=mean + n_std * std
            )
    
    return df

In [16]:
handling_outliers(df, OUTLIER_COLUMNS,thresholds=COLUMN_THRESHOLDS, method='clip')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917.0,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000.0
1456,1457,20,RL,85.0,13175.0,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000.0
1457,1458,70,RL,66.0,9042.0,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500.0
1458,1459,20,RL,68.0,9717.0,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125.0
